In [1]:
import os
import pandas as pd

In [10]:
# read the datadictionary on the third sheet from excel file
datadict = pd.read_excel('CollegeScorecardDataDictionary.xlsx', sheet_name=3)

# retrieve datadictionary information for degrees
deg_cols = datadict[(datadict['dev-category']=='academics') &  (datadict['VARIABLE NAME'].str.contains('PCIP'))][['VARIABLE NAME', 'LABEL']]

# reset index
deg_cols.reset_index(drop=True, inplace=True)

print(deg_cols.shape)
deg_cols.head(1)

(38, 2)


,VARIABLE NAME,LABEL
0,PCIP01,"Agriculture, Agriculture Operations, and Relat..."


In [78]:
df = pd.read_csv('MERGED2018_19_pp.csv', encoding='utf-8', low_memory=False)

new_column_names = { row['VARIABLE NAME']: row['LABEL'] for index, row in deg_cols.iterrows() }
df = df[['UNITID',*deg_cols['VARIABLE NAME'].to_list()]].rename(columns={'UNITID': 'college_id', 'INSTNM': 'name', **new_column_names})

print(df.shape)
df.head(1)

(6806, 39)


,college_id,"Agriculture, Agriculture Operations, and Related Sciences",Natural Resources and Conservation,Architecture and Related Services,"Area, Ethnic, Cultural, Gender, and Group Studies","Communication, Journalism, and Related Programs",Communications Technologies/Technicians and Support Services,Computer and Information Sciences and Support Services,Personal and Culinary Services,Education,...,Public Administration and Social Service Professions,Social Sciences,Construction Trades,Mechanic and Repair Technologies/Technicians,Precision Production,Transportation and Materials Moving,Visual and Performing Arts,Health Professions and Related Programs,"Business, Management, Marketing, and Related Support Services",History
0,100654,0.0394,0.0237,0.0039,0.0,0.0,0.0394,0.0592,0.0,0.071,...,0.0493,0.0355,0.0,0.0,0.0,0.0,0.0237,0.0,0.1578,0.0


In [79]:
college_majors = pd.DataFrame((df.set_index(['college_id'])
     .rename_axis('major', axis=1)
     .stack())).reset_index()

In [80]:
college_majors.shape

(229368, 3)

In [81]:
college_majors[(college_majors['college_id']==100654)].shape

(38, 3)

In [82]:
college_majors[(college_majors['college_id']==100654)&(college_majors[0] > 0)].shape

(21, 3)

In [83]:
college_majors[college_majors[0] > 0].iloc[:,0:2].shape

(49651, 2)

In [84]:
college_majors = college_majors[college_majors[0] > 0].iloc[:,0:2]

In [103]:
# store list of scorecard degrees in CSV file
college_majors.groupby('major')[['major']].nunique().rename(columns={'major': 'count'}).reset_index()[['major']].to_csv('scorecard_majors.csv', sep=',', encoding='utf-8', mode='w', index=False)

In [85]:
college_majors.head(2)

,college_id,major
0,100654,"Agriculture, Agriculture Operations, and Relat..."
1,100654,Natural Resources and Conservation


In [74]:
degrees = pd.read_csv('../kaggle/degrees-that-pay-back.csv', encoding='utf-8', low_memory=False)
degrees = degrees[['Undergraduate Major']].rename(columns={'Undergraduate Major': 'major'})

In [105]:
print(degrees.shape)
degrees.head(2)

(50, 1)


,major
0,Accounting
1,Aerospace Engineering


In [106]:
degrees.groupby('major')[['major']].nunique().rename(columns={'major': 'count'}).reset_index()[['major']].to_csv('kaggle_majors.csv', sep=',', encoding='utf-8', mode='w', index=False)

In [ ]:
for index, row in college_majors.iterrows():
    for major in degrees.major.to_list():
        if(major in row.major):
            print(row.college_id)
    

In [92]:
counter = 0
for index, row in college_majors.iterrows():
    counter += 1
    if counter <=50:
        print('Accounting' in row.major)
    else:
        break

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
